In [1]:
import os 
import numpy as np 
import pandas as pd 
import joblib 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
import seaborn as sns
import plotly.express as px
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
data_frames = []
y = []
for exp in range(1, 2):
    for cls in range(11):
        df = pd.read_excel(
            f'Data/Experiment_{exp}/Class_{cls}/channel_c.xlsx',
            header=None
        )
        # df["experiment"] = exp
        # df["class"] = cls
        data_frames.append(df)
        # data_frames.append(df.iloc[:28].copy())
    y_1 = [i for i in range(11) for _ in range(3)]
    y.append(y_1)

X = pd.concat(data_frames, axis=0, ignore_index=True)
y = np.array(y).flatten()

In [8]:
X.shape, y.shape

((33, 128), (33,))

In [10]:
df = X.copy()
df['y'] = y

In [19]:
description = df.groupby('y').describe()

In [ ]:
df.groupby('y')

## Previous Code

In [ ]:
df = df.dropna(axis=0, how="any").reset_index(drop=True)

In [11]:
def remove_outliers_iqr(df, label_col="y"):
    # Separate features and label
    X = df.drop(columns=[label_col])
    
    # Compute IQR for each feature
    Q1 = X.quantile(0.25)
    Q3 = X.quantile(0.75)
    IQR = Q3 - Q1

    # Keep only rows where all features are within IQR bounds
    mask = ~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)
    
    return df[mask].reset_index(drop=True)

In [6]:
df_clean = remove_outliers_iqr(df, label_col="y")
df_clean = df_clean.dropna(axis=0, how="any").reset_index(drop=True)
print(df.shape, "->", df_clean.shape)

(1400, 129) -> (528, 129)


In [7]:
df_clean[df_clean.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,y


In [30]:
df_clean = df  

In [31]:
# Convert data to NumPy arrays
X = df_clean.drop(columns=['y']).to_numpy()
y = df_clean['y'].to_numpy()

In [ ]:
# X = df.drop(columns=['y']).to_numpy()
# y = df['y'].to_numpy()

In [32]:
# # Normalize the data using StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [33]:
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X, y)
df_lda = pd.DataFrame(data=X_lda, columns=['C1', 'C2'])
df_lda['target'] = y
df_lda = pd.DataFrame(data=X_lda, columns=['C1', 'C2'])
df_lda['target'] = y 
df_lda['target'] = df_lda['target'].astype(str)

fig = px.scatter(df_lda, x='C1', y='C2', color='target', 
                 color_discrete_sequence=px.colors.qualitative.Set1,
                 title='Inter-ring Excitation: Projection onto First and Second LDA Components',
                 labels={'target': 'Classes'},
                 hover_data={'C1': True, 'C2': True, 'target': True})
fig.update_layout(legend_title_text='Classes')
fig.show()

Train

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [20]:
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 64, 32), 
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

In [21]:
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(64, 64, 32), max_iter=500, random_state=42)

In [17]:
import joblib

joblib.dump(mlp, "model/mlp_model_3.pkl")

loaded_mlp = joblib.load("model/mlp_model_3.pkl")

In [22]:
y_pred = mlp.predict(X_test)

In [23]:
mlp.predict(X_test[:1])[0]

np.int64(1)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Confusion Matrix")
plt.show()

In [28]:
report = classification_report(y_test, y_pred)
print("Classification Report:\n")
print(report)

Classification Report:

              precision    recall  f1-score   support

           0       0.70      0.93      0.80        28
           1       0.64      0.32      0.42        22
           2       0.50      0.43      0.46        21
           3       0.47      0.67      0.55        21
           4       0.75      0.30      0.43        20
           5       0.68      0.95      0.79        22
           6       0.91      0.83      0.87        24
           7       0.91      0.84      0.88        25
           8       0.79      0.86      0.83        22
           9       0.91      0.95      0.93        21

    accuracy                           0.72       226
   macro avg       0.73      0.71      0.70       226
weighted avg       0.73      0.72      0.71       226



In [30]:
classes = np.unique(y_test)
y_test_bin = label_binarize(y_test, classes=classes)
y_pred_bin = label_binarize(y_pred, classes=classes)

In [32]:
ap_per_class = []
for i in range(len(classes)):
    ap = average_precision_score(y_test_bin[:, i], y_pred_bin[:, i])
    ap_per_class.append(ap)
    print(f"AP for class {classes[i]}: {ap:.4f}")
    
mAP = np.mean(ap_per_class)
print(f"\nMean Average Precision (mAP): {mAP:.4f}")

AP for class 0: 0.6614
AP for class 1: 0.2689
AP for class 2: 0.2674
AP for class 3: 0.3421
AP for class 4: 0.2869
AP for class 5: 0.6511
AP for class 6: 0.7753
AP for class 7: 0.7847
AP for class 8: 0.6970
AP for class 9: 0.8702

Mean Average Precision (mAP): 0.5605
